**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Jan 14 11:33:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%%capture
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

**1) Clone the Darknet**



In [5]:
%%capture
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [6]:
%%capture
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [13]:
!echo "Brasil6G" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [12]:
%%capture
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [14]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/9.txt          
  inflating: data/obj/7.txt          
  inflating: data/obj/50.txt         
  inflating: data/obj/5.txt          
  inflating: data/obj/49.txt         
  inflating: data/obj/47.txt         
  inflating: data/obj/46.txt         
  inflating: data/obj/45.txt         
  inflating: data/obj/44.txt         
  inflating: data/obj/43.txt         
  inflating: data/obj/42.txt         
  inflating: data/obj/41.txt         
  inflating: data/obj/40.txt         
  inflating: data/obj/4.txt          
  inflating: data/obj/39.txt         
  inflating: data/obj/38.txt         
  inflating: data/obj/37.txt         
  inflating: data/obj/36.txt         
  inflating: data/obj/33.txt         
  inflating: data/obj/30.txt         
  inflating: data/obj/0.txt          
  inflating: data/obj/13.txt         
  inflating: data/obj/19.txt         
  inflating: data/obj/23.txt         
  inflating: 

In [15]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

20 0.502344 0.495833 0.989062 0.986111

['20', '0.502344', '0.495833', '0.989062', '0.986111']
0 data/obj/30.txt
0 0.502344 0.495833 0.989062 0.986111
16 0.439844 0.413889 0.151562 0.183333

['16', '0.439844', '0.413889', '0.151562', '0.183333']
1 data/obj/9.txt
0 0.439844 0.413889 0.151562 0.183333
16 0.550781 0.151389 0.132812 0.286111

['16', '0.550781', '0.151389', '0.132812', '0.286111']
1 data/obj/9.txt
0 0.550781 0.151389 0.132812 0.286111
16 0.364063 0.554167 0.103125 0.130556

['16', '0.364063', '0.554167', '0.103125', '0.130556']
1 data/obj/9.txt
0 0.364063 0.554167 0.103125 0.130556
16 0.273438 0.684722 0.096875 0.175000

['16', '0.273438', '0.684722', '0.096875', '0.175000']
1 data/obj/9.txt
0 0.273438 0.684722 0.096875 0.175000
16 0.385937 0.843056 0.134375 0.236111

['16', '0.385937', '0.843056', '0.134375', '0.236111']
1 data/obj/9.txt
0 0.385937 0.843056 0.134375 0.236111
16 0.489063 0.687500 0.115625 0.225000

['16', '0.489063', '0.687500', '0.115625', '0.225000']
1 da

In [16]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/11.jpg', 'data/obj/29.jpg', 'data/obj/31.jpg', 'data/obj/49.jpg', 'data/obj/21.jpg', 'data/obj/47.jpg', 'data/obj/19.jpg', 'data/obj/0.jpg', 'data/obj/14.jpg', 'data/obj/39.jpg', 'data/obj/17.jpg', 'data/obj/8.jpg', 'data/obj/35.jpg', 'data/obj/42.jpg', 'data/obj/15.jpg', 'data/obj/38.jpg', 'data/obj/43.jpg', 'data/obj/3.jpg', 'data/obj/36.jpg', 'data/obj/46.jpg', 'data/obj/40.jpg', 'data/obj/45.jpg', 'data/obj/5.jpg', 'data/obj/12.jpg', 'data/obj/24.jpg', 'data/obj/9.jpg', 'data/obj/37.jpg', 'data/obj/27.jpg', 'data/obj/25.jpg', 'data/obj/7.jpg', 'data/obj/23.jpg', 'data/obj/16.jpg', 'data/obj/50.jpg', 'data/obj/6.jpg', 'data/obj/22.jpg', 'data/obj/28.jpg', 'data/obj/13.jpg', 'data/obj/1.jpg', 'data/obj/18.jpg', 'data/obj/20.jpg', 'data/obj/34.jpg', 'data/obj/30.jpg', 'data/obj/32.jpg', 'data/obj/33.jpg', 'data/obj/26.jpg', 'data/obj/44.jpg', 'data/obj/2.jpg', 'data/obj/41.jpg', 'data/obj/10.jpg', 'data/obj/48.jpg', 'data/obj/4.jpg']


In [17]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

A saída de streaming foi truncada nas últimas 5000 linhas.
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/34.txt 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.553175), count: 2, class_loss = 0.502155, iou_loss = 0.480873, total_loss = 0.983028 
 total_bbox = 374124, rewritten_bbox = 0.000000 % 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/35.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/26.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/6.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/35.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/48.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/35.txt 
Can't open label file. (This can be normal only if you use MSCOCO): data/obj/34.txt 
v3 (mse loss, Normalizer: (iou: 0.75, obj: